In [5]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import pyautogui
import time

# CSV 파일 경로
FILE_PATH = r'D:\시설\정지\일일정지\20250114_sp별정지사유등록문자발송2.cs'

# 전역 변수
start_position = None  # 첫 번째 클릭 위치 (등록 시작점)
registered_numbers = []  # 등록 완료된 번호 리스트
start_index = 0  # 현재 등록 시작 인덱스 (100개 단위)

# 딜레이 설정 (초)
DELAY_BETWEEN_INPUTS = 0.05  # 각 번호 입력 후 딜레이
DELAY_BETWEEN_TABS = 0.1  # 탭키 입력 후 딜레이


def load_data():
    """
    CSV 파일에서 데이터를 로드하고 번호를 11자리로 맞춤
    """
    try:
        data = pd.read_csv(FILE_PATH, encoding='euc-kr')
        data['문자발송용'] = data['문자발송용'].astype(str).apply(lambda x: x.zfill(11))
        data.reset_index(drop=True, inplace=True)
        return data
    except FileNotFoundError:
        messagebox.showerror("파일 오류", f"CSV 파일을 찾을 수 없습니다: {FILE_PATH}")
        return pd.DataFrame()
    except Exception as e:
        messagebox.showerror("데이터 오류", f"데이터를 불러오는 중 오류가 발생했습니다: {e}")
        return pd.DataFrame()


def update_tables(data):
    """
    테이블 업데이트: 불러온 번호와 등록 완료된 번호를 업데이트
    """
    # 불러온 번호 테이블 갱신
    left_table.delete(*left_table.get_children())
    remaining_numbers = data[~data['문자발송용'].isin(registered_numbers)]
    for idx, row in remaining_numbers.iterrows():
        left_table.insert("", "end", values=(row.name + 1, row['문자발송용']))

    # 등록 완료된 번호 테이블 갱신
    registered_table.delete(*registered_table.get_children())
    for idx, number in enumerate(registered_numbers, start=1):
        registered_table.insert("", "end", values=(idx, number))


def execute_registration(number):
    """
    번호 등록 실행: 지정된 좌표에 값을 입력하고 탭키로 이동
    """
    try:
        pyautogui.write(number)
        time.sleep(DELAY_BETWEEN_INPUTS)  # 딜레이 최소화
        pyautogui.press("tab")  # 탭키로 다음 입력란으로 이동
        time.sleep(DELAY_BETWEEN_TABS)  # 탭키 이동 후 딜레이

        registered_numbers.append(number)
        print(f"번호 '{number}' 등록 완료")
    except Exception as e:
        messagebox.showerror("등록 오류", f"번호 '{number}' 등록 실패: {e}")


def manual_registration(data):
    """
    수동 등록: 좌표 설정 후 100개 단위로 입력
    """
    global start_position, start_index

    if not start_position:
        messagebox.showinfo("좌표 설정", "5초 안에 '받는 사람' 입력란을 클릭하세요.")
        time.sleep(5)  # 5초 대기
        start_position = pyautogui.position()

        if not start_position:
            messagebox.showwarning("좌표 설정 실패", "입력란 좌표를 설정하지 못했습니다. 다시 시도하세요.")
            return

    x, y = start_position

    end_index = start_index + 100
    if start_index >= len(data):
        messagebox.showinfo("완료", "더 이상 등록할 번호가 없습니다.")
        return

    pyautogui.click(x=x, y=y)
    time.sleep(0.2)

    for idx, row in data[start_index:end_index].iterrows():
        execute_registration(row['문자발송용'])

    start_index = end_index
    update_tables(data)
    messagebox.showinfo("완료", f"{start_index - 99}번부터 {start_index}번까지 등록이 완료되었습니다!")


def create_gui(data):
    """
    GUI 생성 및 구성
    """
    global left_table, registered_table

    root = tk.Tk()
    root.title("휴대폰 번호 등록 시스템")
    root.geometry("1000x700")
    root.configure(bg="#f8f9fa")

    # 타이틀
    title_label = tk.Label(
        root, text="휴대폰 번호 등록 시스템", font=("Arial", 20, "bold"), bg="#343a40", fg="#ffffff", pady=10
    )
    title_label.pack(fill=tk.X)

    # 테이블 프레임
    table_frame = tk.Frame(root, bg="#f8f9fa")
    table_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

    # 왼쪽 테이블 (불러온 번호)
    left_frame = tk.Frame(table_frame, bg="#ffffff", bd=2, relief=tk.RIDGE)
    left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10)

    left_label = tk.Label(left_frame, text="불러온 휴대폰번호", font=("Arial", 14, "bold"), bg="#ffffff")
    left_label.pack(pady=10)

    left_scroll = tk.Scrollbar(left_frame, orient="vertical")
    left_table = ttk.Treeview(
        left_frame,
        columns=("no", "number"),
        show="headings",
        height=20,
        yscrollcommand=left_scroll.set,
    )
    left_scroll.config(command=left_table.yview)
    left_scroll.pack(side=tk.RIGHT, fill=tk.Y)

    left_table.heading("no", text="No")
    left_table.heading("number", text="휴대폰번호")
    left_table.column("no", width=50, anchor="center")
    left_table.column("number", width=250, anchor="center")
    left_table.pack(fill=tk.BOTH, expand=True, padx=10)

    # 오른쪽 테이블 (등록 완료 번호)
    right_frame = tk.Frame(table_frame, bg="#ffffff", bd=2, relief=tk.RIDGE)
    right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10)

    right_label = tk.Label(right_frame, text="등록완료된 휴대폰번호", font=("Arial", 14, "bold"), bg="#ffffff")
    right_label.pack(pady=10)

    right_scroll = tk.Scrollbar(right_frame, orient="vertical")
    registered_table = ttk.Treeview(
        right_frame,
        columns=("no", "number"),
        show="headings",
        height=20,
        yscrollcommand=right_scroll.set,
    )
    right_scroll.config(command=registered_table.yview)
    right_scroll.pack(side=tk.RIGHT, fill=tk.Y)

    registered_table.heading("no", text="No")
    registered_table.heading("number", text="휴대폰번호")
    registered_table.column("no", width=50, anchor="center")
    registered_table.column("number", width=250, anchor="center")
    registered_table.pack(fill=tk.BOTH, expand=True, padx=10)

    # 하단 버튼
    button_frame = tk.Frame(root, bg="#f8f9fa")
    button_frame.pack(fill=tk.X, pady=10)

    execute_button = tk.Button(
        button_frame,
        text="실행",
        command=lambda: manual_registration(data),
        font=("Arial", 14, "bold"),
        bg="#28a745",
        fg="#ffffff",
        width=15,
    )
    execute_button.pack(side=tk.LEFT, padx=10)

    exit_button = tk.Button(
        button_frame,
        text="종료",
        command=root.destroy,
        font=("Arial", 14, "bold"),
        bg="#dc3545",
        fg="#ffffff",
        width=15,
    )
    exit_button.pack(side=tk.RIGHT, padx=10)

    update_tables(data)
    root.mainloop()


if __name__ == "__main__":
    data = load_data()
    create_gui(data)

번호 '01071954890' 등록 완료
번호 '01028891608' 등록 완료
번호 '01032280237' 등록 완료
번호 '01040048898' 등록 완료
번호 '01042613797' 등록 완료
번호 '01032418520' 등록 완료
번호 '01088691347' 등록 완료
번호 '01044302393' 등록 완료
번호 '01058805501' 등록 완료
번호 '01037165315' 등록 완료
번호 '01020805063' 등록 완료
번호 '01071638361' 등록 완료
번호 '01099030707' 등록 완료
번호 '01043699112' 등록 완료
번호 '01044575009' 등록 완료
번호 '01051956329' 등록 완료
번호 '01092090419' 등록 완료
번호 '01091888887' 등록 완료
번호 '01053514624' 등록 완료
번호 '01099791104' 등록 완료
번호 '01079005280' 등록 완료
번호 '01027631951' 등록 완료
번호 '01034480466' 등록 완료
번호 '01086344514' 등록 완료
번호 '01066497514' 등록 완료
번호 '01054737057' 등록 완료
번호 '01099417372' 등록 완료
번호 '01043931053' 등록 완료
번호 '01071251116' 등록 완료
번호 '01043250811' 등록 완료
번호 '01087418315' 등록 완료
번호 '01067367431' 등록 완료
번호 '01076850112' 등록 완료
번호 '01031730219' 등록 완료
번호 '01067234755' 등록 완료
번호 '01051299293' 등록 완료
번호 '01044456171' 등록 완료
번호 '01080102259' 등록 완료
번호 '01026621073' 등록 완료
번호 '01089193323' 등록 완료
번호 '01062511326' 등록 완료
번호 '01046936866' 등록 완료
번호 '01022703685' 등록 완료
번호 '0108867

In [ ]:
# 정지 voc병합
import pandas as pd
import glob
import os

# 폴더 경로 설정
folder_path = r'D:\시설\정지\정지VOC'

# 모든 .xls 및 .xlsx 파일 경로 가져오기
all_files = glob.glob(folder_path + "/*.xls") + glob.glob(folder_path + "/*.xlsx")

# 병합할 데이터를 저장할 데이터프레임 초기화
all_data = pd.DataFrame()

# 파일 처리
for file in all_files:
    try:
        # 파일 확장자에 따라 처리
        if file.endswith(".xls") or file.endswith(".xlsx"):
            # Excel 파일 처리
            df = pd.read_excel(file, sheet_name=0)  # 첫 번째 시트만 읽기
        else:
            # HTML 형식으로 처리
            df_list = pd.read_html(file)
            df = df_list[0]  # 첫 번째 테이블만 사용
       
        # 데이터가 유효한 경우만 병합
        if not df.empty:
            all_data = pd.concat([all_data, df], ignore_index=True)
            print(f"{file} 데이터 미리보기:\n", df.head())
        else:
            print(f"{file} 데이터가 비어 있습니다. 건너뜁니다.")
    except Exception as e:
        print(f"파일을 읽는 중 오류 발생: {file}, 오류: {e}")

# 병합된 데이터를 csv 파일로 저장 (CP949 인코딩 적용)
output_path = os.path.join(folder_path, "정지VOC분석병합2023-20250113.csv")
if not all_data.empty:
    all_data.to_csv(output_path, index=False, encoding='cp949')
    print(f"병합된 파일이 {output_path}에 저장되었습니다.")
else:
    print("병합된 데이터가 비어 있습니다. 파일 형식과 데이터를 확인하세요.")